# Assignment B - RhoA activation and inhibition analysis using FRET
### Author: Theodoros Foskolos
### Date: June, 2023

In [43]:
import os
import napari
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import cv2
import numpy.ma as ma
from scipy import ndimage
from skimage import exposure, transform, io, img_as_ubyte
import tifffile as tif
from PIL import Image
from skimage.registration import phase_cross_correlation
from skimage.transform import AffineTransform, warp
from scipy.ndimage import fourier_shift
import imageio

## Import and inspect the image data

In [28]:
#Start coding here
filepath ='IPQDA_23_ASS_B_data/W47-SGFP2-mScarlet-I-01-1_2channels.tif'
image = tif.imread(filepath)
#End coding here
image.shape

(70, 2, 260, 348)

## Extract both the donor and acceptor channels (donor = channel 1 and acceptor = channel 2)

In [29]:
# Start coding here
donor_channel = image[:,0,:,:]
acceptor_channel = image[:,1,:,:]
donor_channel.shape
# End coding here

(70, 260, 348)

## Calclate and plot the z-projections (use mean) of both channels

# Question 4: What is the purpose of calculating the mean of the z-projection of each image stack: The purpose of calculating the Z-Projection of each image stack is to get the average intensity of the stacks as a single image representing all the Z axis. This procedure helps us detect the background ROI that we will subtract later and use it to reduce noise

In [30]:
# Start coding here
z_project_donor = np.mean(donor_channel, axis=0)
z_project_acceptor = np.mean(acceptor_channel, axis=0)
z_project_acceptor.shape
# End coding here

(260, 348)

In [ ]:
#%matplotlib inline, if i use that it disturbs the  TkAGG
# Plot z-projections
plt.imshow(z_project_donor)
plt.show()

In [ ]:
plt.imshow(z_project_acceptor)
plt.show()

## Extract background mask (region with no cell(s))
Refer to Tutorial 2 for steps to create mask of region of interest (ROI)

I tried to do it based on the tutorial 2 by drawing the roi polylines on the background and use that as a mask but it seemed to miss something, and tutorial doesnt contain much information on the opencv method, only napari which i encountered several issues. Simple thresholding for both channels based on the mean of the mean projections worked fine and separated the background from the cells quite well.

In [ ]:
"""# Make donor and acceptor RGB format
Donor_RGB = cv2.cvtColor(z_project_donor, cv2.COLOR_BGR2RGB)
Acceptor_RGB = cv2.cvtColor(z_project_acceptor, cv2.COLOR_BGR2RGB)
Donor_RGB.shape
# End coding here"""


In [ ]:
"""# Pick 3 points of the background to create a roi for donor and then for acceptor
plt.imshow(Donor_RGB)
points_donor = np.array(plt.ginput(4))
plt.draw()
plt.show()"""

In [ ]:
"""plt.imshow(Acceptor_RGB)
points_accept = np.array(plt.ginput(4))
plt.draw()
plt.show()"""

In [ ]:
"""#Convert the selected points to integer coordinates:
roi_points_donor =np.int32([points_donor])
roi_points_accept = np.int32([points_accept])"""


In [ ]:
"""#Draw the ROI on the image using OpenCV:
background_mask_Donor = cv2.polylines(Donor_RGB,roi_points_donor, isClosed=True, color=(0, 255, 0), thickness=1)
background_mask_Accept = cv2.polylines(Acceptor_RGB,roi_points_accept, isClosed=True, color=(0, 255, 0), thickness=1)"""


In [ ]:
"""#Display the image with the ROI:
plt.figure()
plt.imshow(Acceptor_RGB)
plt.show()"""

In [ ]:
"""#Display the image with the ROI:
plt.figure()
plt.imshow(Donor_RGB)
plt.show()"""

In [31]:
threshold_donor = np.mean(z_project_donor)
threshold_acceptor = np.mean(z_project_acceptor)

In [32]:
# if intensity in both images is below the thresholds -> background = true
background_mask_Donor = (z_project_donor < threshold_donor)
background_mask_Accept = (z_project_acceptor < threshold_acceptor)
#background_mask.shape
plt.imshow(background_mask_Accept)
plt.show()

## Calculate average background intensity and also the standard deviation for all time frames and both channels. Plot them vs time.

In [33]:
# Mean and std dev calculation of the z projections background
mean_donor_z = np.mean(background_mask_Donor)
mean_acceptor_z = np.mean(background_mask_Accept)
stddev_donor_z = np.std(background_mask_Donor)
stddev_acceptor_z = np.std(background_mask_Accept)

mean_donor = []
mean_acceptor = []
stddev_donor = []
stddev_acceptor = []

# Same, for each time point of the image in both channels based on the ROIs i selected before for the background.
for t in range(image.shape[0]):
    """ #This part is only for, ROI selection by hand, which seem to not work properly
    # Extract the background region using the ROI points, also ensure that the selected points for the ROI are within the valid range of the acceptor channel image dimensions
    background_donor = donor_channel[t][
        np.clip(roi_points_donor[:, 1], 0, donor_channel.shape[1] - 1),
        np.clip(roi_points_donor[:, 0], 0, donor_channel.shape[2] - 1)
    ]
    background_acceptor = acceptor_channel[t][
        np.clip(roi_points_accept[:, 1], 0, acceptor_channel.shape[1] - 1),
        np.clip(roi_points_accept[:, 0], 0, acceptor_channel.shape[2] - 1)
    ]"""
    # Get the background for frames
    background_donor = donor_channel[t][background_mask_Donor]
    background_acceptor = acceptor_channel[t][background_mask_Accept]

    # Mean and standard deviation for the backgroundd and add it to the list for each timepoint
    mean_donor.append(np.mean(background_donor))
    mean_acceptor.append(np.mean(background_acceptor))
    stddev_donor.append(np.std(background_donor))
    stddev_acceptor.append(np.std(background_acceptor))

# End coding here

In [34]:
"""# Plot average intensities of background and standard deviations against time
time_frames = np.arange(donor_channel.shape[0])

plt.figure(figsize=(10, 6))
plt.errorbar(time_frames, mean_donor, yerr=stddev_donor, label='Donor Channel')
plt.errorbar(time_frames, mean_acceptor, yerr=stddev_acceptor, label='Acceptor Channel')
plt.xlabel('Time')
plt.ylabel('Intensity')
plt.title('Average Intensity and Standard Deviation vs Time')
plt.legend()
plt.grid(True)
plt.show()

"""

"# Plot average intensities of background and standard deviations against time\ntime_frames = np.arange(donor_channel.shape[0])\n\nplt.figure(figsize=(10, 6))\nplt.errorbar(time_frames, mean_donor, yerr=stddev_donor, label='Donor Channel')\nplt.errorbar(time_frames, mean_acceptor, yerr=stddev_acceptor, label='Acceptor Channel')\nplt.xlabel('Time')\nplt.ylabel('Intensity')\nplt.title('Average Intensity and Standard Deviation vs Time')\nplt.legend()\nplt.grid(True)\nplt.show()\n\n"

In [35]:
#Plotting
plt.plot(mean_donor,label='donor')
plt.plot(mean_acceptor,label='acceptor')
# Set the labels
plt.xlabel('time')
plt.ylabel('intensity')
# Set the legend
plt.legend()
plt.show()
plt.close()

plt.plot(stddev_donor,label='donor')
plt.plot(stddev_acceptor,label='acceptor')
plt.xlabel('time')
plt.ylabel('intensity')

# Set the legend
plt.legend()
plt.show()

## Image processing steps

In [36]:
#Initializing parameters

# Define a smoothing kernel
ks = (3,3) # kernel size should be a tuple
kernel = np.ones(ks) / np.prod(ks) # create box filter kernel

# Initialize output array
corrected_frames = np.zeros_like(image,dtype=np.float64)

In [37]:
#Preparing the image data
# Convert pixel type to float
image = image.astype(np.float64)
background_mask_Accept = background_mask_Accept.astype(np.float64)
background_mask_Donor = background_mask_Donor.astype(np.float64)

""" # this part also only for the hand picked ROI of background
# I encountered this problem that the RGB has 3 channels and i can use it for subtraction and i dont know how so i make it gray
mask_gray_don = cv2.cvtColor(background_mask_Donor, cv2.COLOR_RGB2GRAY)
mask_gray_accept = cv2.cvtColor(background_mask_Accept, cv2.COLOR_RGB2GRAY)

mask_gray_don.shape

# Channels extraction"""

donor_channel = image[:,0,:,:]
acceptor_channel = image[:,1,:,:]

In [38]:
#Now they begin - image processing

for t in range(image.shape[0]):
    
    #######################################
    ##Background subtraction
    #Start coding here
    donor_channel[t, :, :] = donor_channel[t, :, :] - background_mask_Donor
    acceptor_channel[t, :, :] = acceptor_channel[t, :, :] - background_mask_Accept

    #######################################
    ##Image registration
    #Calculate the shift between the two images
    shift, error, diffphase = phase_cross_correlation(donor_channel[t, :, :], acceptor_channel[t, :, :])


    #Create an affine transform object with the shift
    tform = AffineTransform(translation=(-shift[1], -shift[0]))

    #Apply the transformation to the image 
    corrected_image = warp(acceptor_channel[t, :, :], tform.inverse)

    # Replace the original channel with the corrected channel
    acceptor_channel[t, :, :] = corrected_image

    #######################################
    ##Image processing steps to reduce noise
    ##Smoothing (hint: use the created kernel)
    donor_channel[t, :, :] = ndimage.convolve(donor_channel[t, :, :], kernel)
    acceptor_channel[t, :, :] = ndimage.convolve(acceptor_channel[t, :, :], kernel)

    #Thresholding so that pixels with low signal are assigned with np.nan. 
    #Use the background mean and standard deviation intensity in this step to form a threshold value
    t_donor = mean_donor[t] +  stddev_donor[t]
    t_acceptor = mean_acceptor[t] +  stddev_donor[t]
    donor_channel[t, :, :][donor_channel[t, :, :] < t_donor] = np.nan
    acceptor_channel[t, :, :][acceptor_channel[t, :, :] < t_acceptor] = np.nan
    #Well done. Let's keep the corrected frames
    corrected_frames[t, 1, :, :] = acceptor_channel[t, :, :]
    corrected_frames[t, 0, :, :] = donor_channel[t, :, :]
    print(f'{t} of {image.shape[0]}')
        

    # End coding here

0 of 70
1 of 70
2 of 70
3 of 70
4 of 70
5 of 70
6 of 70
7 of 70
8 of 70
9 of 70
10 of 70
11 of 70
12 of 70
13 of 70
14 of 70
15 of 70
16 of 70
17 of 70
18 of 70
19 of 70
20 of 70
21 of 70
22 of 70
23 of 70
24 of 70
25 of 70
26 of 70
27 of 70
28 of 70
29 of 70
30 of 70
31 of 70
32 of 70
33 of 70
34 of 70
35 of 70
36 of 70
37 of 70
38 of 70
39 of 70
40 of 70
41 of 70
42 of 70
43 of 70
44 of 70
45 of 70
46 of 70
47 of 70
48 of 70
49 of 70
50 of 70
51 of 70
52 of 70
53 of 70
54 of 70
55 of 70
56 of 70
57 of 70
58 of 70
59 of 70
60 of 70
61 of 70
62 of 70
63 of 70
64 of 70
65 of 70
66 of 70
67 of 70
68 of 70
69 of 70


# Question 5 : The threshold value is used to distinguish between pixels with high signal, which reflect the intended characteristics or objects in the image, and pixels with low signal, which are more likely to be influenced by noise. We can determine a cutoff point below which the pixel values are regarded as noise or background by establishing a threshold based on the background mean and standard deviation intensities. The background standard deviation intensity shows the variety or spread of intensity values in the background, whereas the background mean intensity estimates the average intensity level in the background region. In order to denote insignificance or noise, pixels having intensity levels below a particular threshold (ex. threshold = mean_donor[t] + stddev_donor[t]. This means that the threshold is set as the mean plus the standard deviation) are presumed to be part of the background and are given the value np.NAN. In general this procedure helps us  reduce noise in the subsequent analysis, focusing on the meaningful signal (our 4 cells) and getting quality results.

# Question 6 : In the line with the AffineTransform function, we create an object with a translation transformation that shifts the image in the opposite direction by the specified values, in general based on skimage it allows you to define various types of transformations including translation, rotation, scaling, and shearing.

In [47]:
# PLot
plt.imshow(corrected_frames[12, 0,:,:])
plt.show()

In [39]:
# Calculate FRET ratios. R = Acceptor/Donor
# Start coding here
#ratio =  acceptor_channel / donor_channel
ratio =  corrected_frames[:, 1,:,:] / corrected_frames[:, 0,:,:]

# End coding here

#Plot a layer
plt.imshow(ratio[2,:,:])
plt.show()

In [46]:
# FRET ratio across all pixels for each time-point, ignoring the NAN values
mean_ratio = np.nanmean(ratio, axis=(1, 2))

# Plot the mean FRET ratio over time
plt.plot(mean_ratio)
plt.xlabel('Time')
plt.ylabel('Mean FRET Ratio')
plt.title('Mean FRET Ratio Over Time')
plt.show()

# Apply LUT, transform the image to 8bit and export as Gif

In [45]:
# Define the custom LUT
# Hot LUT:
lut_hot = np.zeros((256, 3), dtype=np.uint8)
lut_hot[:, 0] = np.arange(256)
lut_hot[:, 1] = np.arange(256) // 2
lut_hot[:, 2] = 0


# Create a list to store frames of FRET ratio images
frames = []

# Iterate over the FRET ratio images and convert them to PIL Image objects
for i in range(len(ratio)):
    # Convert the FRET ratio image to 8-bit using img_as_ubyte
    ratio_8bit = img_as_ubyte(ratio[i])

    # Apply the custom LUT to the 8-bit image
    ratio_lut = lut_hot[ratio_8bit]

    # Convert the LUT image array to a PIL Image object
    ratio_image = Image.fromarray(ratio_lut)

    # Create a new figure for each frame
    plt.figure()

    # Show the FRET ratio image
    plt.imshow(ratio_image, cmap='gray')
    plt.axis('off')

    # Add frame number and text annotations
    plt.text(20, 20, f'Frame {i}', color='white', fontsize=5, weight='bold', backgroundcolor='black')

    if i >= 15 and i < 45:
        plt.text(10, 10, 'Histamine Activation', color='white', fontsize=8, weight='bold', backgroundcolor='black')
    elif i >= 45:
        plt.text(10, 10, 'Mepyramine Inhibition', color='white', fontsize=8, weight='bold', backgroundcolor='black')

    # Save the figure
    plt.savefig('tmp.png', bbox_inches='tight', pad_inches=0)
    plt.close()

    # Open the temporary image and convert it to a PIL Image object
    temp_image = Image.open('tmp.png')

    # Append the PIL Image object to the frames list
    frames.append(np.array(temp_image))

# file path
output_path = "fret_ratios.gif"

# Save the frames as an animated GIF
imageio.mimsave(output_path, frames, duration=200, loop=0)
os.remove("tmp.png")